# Modules

In [63]:
import pandas as pd
import numpy as np
import re

# Variables

In [5]:
input_folder = '../data/raw_data/popdata/'
output_folder = '../data/processed_data/popdata'

# Process data

In [84]:
df_raw_dane_data = pd.read_excel(f'{input_folder}/Edades_Simples_1985-2020.xls', sheet_name='Total')

df_raw_dane_data.to_csv('tmp_raw_dane_data', index=False)
with open('tmp_raw_dane_data', 'r') as file:
    data = file.readlines()

In [168]:
line_increment = 19
init_line = 11
pivot_line = init_line

years     = [line.strip().split(',') for line in data[init_line-2 : init_line - 1]]
years     = np.sort(np.unique(years))
correct_column_name = np.append(['Codigo', 'Grupos de edad'], np.array([(f'Total_{year}', f'Hombres_{year}', f'Mujeres_{year}') for year in years[years!='']]).flatten())

pattern = r'^([^,]+),'
df_pop = pd.DataFrame({})
while True:
    match  = re.search(pattern, data[pivot_line])
    codigo = match.group(1)
    ##print(codigo)
    if 'DANE' in codigo:
        break

    data_rows = [line.strip().split(',') for line in data[pivot_line   : pivot_line + line_increment]]
    columns   = [line.strip().split(',') for line in data[pivot_line-1 : pivot_line]]

    df_ = pd.DataFrame(data_rows[1:], columns=correct_column_name).melt(id_vars=["Codigo", "Grupos de edad"]).rename(columns={'Grupos de edad':'AgeGroup',                                                                                                                    
                                                                                                                              'Codigo':'Code',
                                                                                                                              'value':'Pop'})
    df_['Year']     = df_.variable.str[-4:]
    df_['Gender']   = df_.variable.str[:-5].replace('Mujeres', 'Female').replace('Hombres', 'Male')
    df_['AgeGroup'] = df_['AgeGroup'].replace('80 Y MÁS', '80-above')
    df_['Code']     = codigo
    del df_['variable']

    df_pop = pd.concat([df_pop, df_]).reset_index(drop=True)

    pivot_line += line_increment

00
05
08
11
13
15
17
18
19
20
23
25
27
41
44
47
50
52
54
63
66
68
70
73
76
81
85
86
88
91
94
95
97
99
05001
05002
05004
05021
05030
05031
05034
05036
05038
05040
05042
05044
05045
05051
05055
05059
05079
05086
05088
05091
05093
05101
05107
05113
05120
05125
05129
05134
05138
05142
05145
05147
05148
05150
05154
05172
05190
05197
05206
05209
05212
05234
05237
05240
05250
05264
05266
05282
05284
05306
05308
05310
05313
05315
05318
05321
05347
05353
05360
05361
05364
05368
05376
05380
05390
05400
05411
05425
05440
05467
05475
05480
05483
05490
05495
05501
05541
05543
05576
05579
05585
05591
05604
05607
05615
05628
05631
05642
05647
05649
05652
05656
05658
05659
05660
05664
05665
05667
05670
05674
05679
05686
05690
05697
05736
05756
05761
05789
05790
05792
05809
05819
05837
05842
05847
05854
05856
05858
05861
05873
05885
05887
05890
05893
05895
08001
08078
08137
08141
08296
08372
08421
08433
08436
08520
08549
08558
08560
08573
08606
08634
08638
08675
08685
08758
08770
08832
08849
11001
1300

In [169]:
df_pop = df_pop[df_pop['AgeGroup'] != 'Total']
df_pop[['MinAge', 'MaxAge']] = df_pop['AgeGroup'].str.split('-', expand=True)
df_pop['MaxAge'] = df_pop['MaxAge'].replace('above', '200')

In [177]:
df_pop[df_pop.Code == '05']

,Code,AgeGroup,Pop,Year,Gender,MinAge,MaxAge
1945,05,0-4,498675,1985,Total,0,4
1946,05,5-9,463696,1985,Total,5,9
1947,05,10-14,429733,1985,Total,10,14
1948,05,15-19,459648,1985,Total,15,19
1949,05,20-24,446664,1985,Total,20,24
...,...,...,...,...,...,...,...
3883,05,60-64,168199,2020,Female,60,64
3884,05,65-69,128023,2020,Female,65,69
3885,05,70-74,93129,2020,Female,70,74
3886,05,75-79,62005,2020,Female,75,79


In [175]:
df_pop.to_csv(f'{output_folder}/colombia_population_data.csv', index=False)